In [71]:
# Load textures from freesound

FREESOUND_API_KEY = '75CqcxRYltGuKSnEPzhSK2TSaiP30YFHyg2uhBhb'  # Please replace by your own Freesound API key
OGG_PATH = 'oggs'  # Place where to store the downloaded diles. Will be relative to the current folder.
WAV_PATH = "wavs"
DAC_PATH = "testdata"
DATAFRAME_FILENAME = 'testdata/dac-train.xlsx'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store
FREESOUND_QUERIES = [
    # {
    #     'query': 'Flute',
    #     'filter': 'pack:"Flute single notes (played badly)"',
    #     'num_results': 50,
    # },

    {
        'query': 'Sax Soprano',
        'filter': 'pack:"Sax Soprano single notes"',
        'num_results': 500,
    },
    {
        'query': 'Sax Tenor',
        'filter': 'pack:"Sax Tenor single notes"',
        'num_results': 500,
    },
        {
        'query': 'Cello',
        'filter': 'pack:"Cello single notes"',
        'num_results': 500,
    },
        {
        'query': 'Flute',
        'filter': 'pack:"Flute single notes"',
        'num_results': 500,
    },
    # {
    #     'query': '',
    #     'filter': 'pack:"Flute single notes (played badly)"',
    #     'num_results': 50,
    # },
# {
#         'query': 'Sax Baritone',
#         'filter': "username:MTG",
#         'num_results': 50,
#     },
# {
#         'query': 'Sax Soprano',
#         'filter': "username:MTG",
#         'num_results': 50,
#     },
# {
#         'query': 'Sax Baritone',
#         'filter': "username:MTG",
#         'num_results': 50,
#     },
#     {
#         'query': 'Cello',
#         'filter': "username:MTG",
#         'num_results': 50,
#     },
#     {
#         'query': 'Trumpet',
#         'filter': "username:MTG",
#         'num_results': 50,
#     },
]
import os
import pandas as pd
import numpy as np
import freesound
from tqdm import tqdm
import shutil
import essentia.standard as es
from sklearn.model_selection import train_test_split
import torch
from dac.utils import load_model
from audiotools import AudioSignal
import dac


def encode_audio_files(df, output_dir, model_sr='44khz', model_tag='latest', device="cuda", n_quantizers=4):
    with torch.no_grad():
        # Load the DAC model
        model_path = dac.utils.download(model_type=model_sr)
        model = dac.DAC.load(model_path).to(device)


        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Iterate over all WAV files
        for file_path in tqdm(df.ID, desc="Encoding audio files"):

            wav_path = df[df.ID == file_path]["WAV Path"].values[0]

            # Load the audio file
            audio = AudioSignal(wav_path)

            # Extract the tensor data and move it to the appropriate device
            # audio_tensor = audio.audio_data.to(device)

            # Encode the audio tensor
            codes = model.compress(audio, n_quantizers=n_quantizers)

            # Define the output file path
            output_file = os.path.join(output_dir, file_path + '.dac')

            codes.save(output_file)
            # Save the encoded codes
            # with open(output_file, 'wb') as f:
            #     torch.save(codes, f)

# def create_train_test_dataframes(sound_dict, wav_filenames, sound_list, features, wav_path, dac_folder):

#     df = pd.DataFrame.from_dict(generate_class_mapping(sound_dict, wav_filenames), orient="index", columns=["Class Name"]).reset_index(names="Audio ID")
#     df["Freesound ID"] = df["Audio ID"].apply(lambda x: x.split("_")[0])
#     df["WAV Path"] = df["Audio ID"].apply(lambda x: os.path.join(wav_path, x) + ".wav")
#     df["DAC Path"] = df["Audio ID"].apply(lambda x: x + ".dac")
    
#     sound_features = np.vstack([get_features(sound, features) for sound in sound_list])

#     for i in range(len(features)):
#         df[f"Param {i + 1}"] = sound_features[:, i]

#     train, test = train_test_split(df, test_size=0.2, stratify=df["Class Name"])

#     train.to_excel(os.path.join(dac_folder, "dac-train.xlsx"))
#     test.to_excel(os.path.join(dac_folder, "dac-val.xlsx"))

#     return train, test

def generate_train_test_dataframes(sound_dict, sound_list, wav_path, dac_folder, features):
    df = pd.DataFrame()
    mapping = generate_class_mapping(sound_dict)
    df["ID"] = [sound.id for sound in sound_list]
    df.set_index("ID", inplace=True)
    df["Name"] = [sound.name for sound in sound_list]
    df["WAV Path"] = [os.path.join(wav_path, f"{sound.id}.wav") for sound in sound_list]
    df["Full File Name"] = [os.path.join(f"{sound.id}.dac") for sound in sound_list]
    df["Class Name"] = [mapping[sound.id] for sound in sound_list]

    sound_features = np.vstack([get_features(sound, features) for sound in tqdm(sound_list, desc="Retrieving sound features")])

    for i in range(len(features)):
        df[f"Param {i + 1}"] = sound_features[:, i] 

    #df = df[~df.index.duplicated(keep='first')]
    df2 = pd.DataFrame(columns=df.columns)
    for wav_filename in os.listdir(wav_path):
        sound_id = int(wav_filename.split('_')[0])
        df2.loc[wav_filename[:-4]] = df.loc[sound_id]
        df2.loc[wav_filename[:-4], "Full File Name"] = wav_filename[:-4] + ".dac"
        df2.loc[wav_filename[:-4], "WAV Path"] = df2.loc[wav_filename[:-4], "WAV Path"].replace(str(sound_id), wav_filename[:-4])

    from sklearn.preprocessing import MinMaxScaler
    df2.loc[:, df2.columns.str.startswith("Param")] = MinMaxScaler().fit_transform(df2.loc[:, df2.columns.str.startswith("Param")])
    df2.reset_index(inplace=True, names="ID")

    print(df2)
    # Remove rows containing NaNs
    df2.dropna(inplace=True)
    train, test = train_test_split(df2, test_size=0.2, stratify=df2["Class Name"])

    train.reset_index(drop=True).to_excel(os.path.join(dac_folder, "dac-train.xlsx"))
    test.reset_index(drop=True).to_excel(os.path.join(dac_folder, "dac-val.xlsx"))
    return train, test

def create_sound_list(freesound_queries, freesound_client):
    sounds = sum([query_freesound(query['query'], query['filter'], freesound_client, query['num_results']) for query in freesound_queries],[])
    return sounds

def create_sound_dict(freesound_queries, freesound_client):
    sound_dict = {query["query"]: query_freesound(query['query'], query['filter'], freesound_client, query['num_results']) for query in freesound_queries}
    return sound_dict

def generate_class_mapping(sound_dict):
    id_to_class = dict()
    for class_name, sounds in sound_dict.items():
        for sound in sounds:
            id_to_class[sound.id] = class_name
    return id_to_class
    # filenames_to_class = dict()
    # for filename in wav_filenames:
    #     basename = os.path.basename(filename)[:-4]
    #     sound_id = int(os.path.basename(filename).split('_')[0])
    #     filenames_to_class[basename] = id_to_class[sound_id]
    # return filenames_to_class

def query_freesound(query, filter, freesound_client, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 0,
        page_size = num_results
    )
    return [sound for sound in pager]
    
def retrieve_sound_preview(sound, freesound_client, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, str(sound.id))
    )

def download_sound_previews(sounds, directory, freesound_client):
    """Download the high-quality OGG sound previews of a list of Freesound sound objects to the given directory.
    """
    for sound in tqdm(sounds, desc='Downloading sound previews'):
        retrieve_sound_preview(sound, freesound_client, directory)

def ogg_to_wav(ogg_folder, wav_folder, dur=5):
    total_length = 0
    wav_filenames = list()
    for filename in os.listdir(ogg_folder):
        ogg_path = os.path.join(ogg_folder, filename)
        
        # Change extension to WAV
        wav_path = os.path.join(wav_folder, filename)

        # Load OGG file using Essentia
        audio = es.MonoLoader(filename=ogg_path)()
        
        # Trim to first 5 seconds (Essentia loads audio as a NumPy array with a sample rate)
        sample_rate = 44100  # Default sample rate in Essentia

        total_length += len(audio) / sample_rate
        
        # Trim audio into 5s frames:
        for i in range(0, len(audio), 5 * sample_rate):
            trimmed_audio = audio[i:i + 5 * sample_rate]
            if len(trimmed_audio) < 5 * sample_rate:
                continue
            # Save as WAV. include i right before the extension
            path = wav_path + f"_{i}.wav"
            
            es.MonoWriter(filename=path, format='wav')(trimmed_audio)

            wav_filenames.append(path)
    return wav_filenames, total_length

import time
def get_features(sound: freesound.FreesoundObject, features: list):
    """Get the features of a given sound object.
    """
    feat_vector = list()
    try:
        a = sound.get_analysis()
        feat_vector = list()
        for feature in features:
            feat_vector.append(eval(f"a.{feature}"))

    
    except Exception as e:
        print(e)
        print(f"Feature extraction failed for sound {sound.id}. Retrying in 10 seconds.")
        time.sleep(10)
        feat_vector = get_features(sound, features)
    return feat_vector
    


def force_mkdir(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)


In [51]:
pager = freesound_client.text_search(
        query = FREESOUND_QUERIES[0]['query'],
        filter = FREESOUND_QUERIES[0]['filter'],
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        descriptors = ','.join(FEATURES),
        group_by_pack = 0,
        page_size = 1
    )

In [61]:
freesound_client.get_sound(sound_list[0].id, descriptors=','.join(FEATURES)).analysis.lowlevel.pitch.mean

214.62378643195854

In [64]:
FEATURES = ["lowlevel.pitch.mean",
            "rhythm.beats_count",
            "rhythm.onset_rate",
            "rhythm.beats_loudness.mean",
            "rhythm.beats_loudness.var",
            "sfx.logattacktime.mean",
            ]
def main():
    force_mkdir(OGG_PATH)
    force_mkdir(WAV_PATH)

    freesound_client = freesound.FreesoundClient()
    freesound_client.set_token(FREESOUND_API_KEY)

    sound_list = create_sound_list(FREESOUND_QUERIES, freesound_client)
    sound_dict = create_sound_dict(FREESOUND_QUERIES, freesound_client)
    

    download_sound_previews(sound_list, OGG_PATH, freesound_client)

    wav_filenames, total_length = ogg_to_wav(OGG_PATH, WAV_PATH)

    train, test = generate_train_test_dataframes(sound_dict, sound_list, WAV_PATH, DAC_PATH, FEATURES)

    train_path = os.path.join(DAC_PATH, "dac-train")
    val_path = os.path.join(DAC_PATH, "dac-val")

    force_mkdir(train_path)
    force_mkdir(val_path)

    encode_audio_files(train, train_path, model_sr='44khz', model_tag='latest')
    encode_audio_files(test, val_path, model_sr='44khz', model_tag='latest')

main()



[wav @ 0x508048e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x508048e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x508048e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x508048e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x508048e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x508048e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x508048e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x508048e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

Feature extraction failed for sound 359635. Retrying in 10 seconds.
Feature extraction failed for sound 359635. Retrying in 10 seconds.
Feature extraction failed for sound 359635. Retrying in 10 seconds.
Feature extraction failed for sound 359635. Retrying in 10 seconds.
Feature extraction failed for sound 359635. Retrying in 10 seconds.
Feature extraction failed for sound 359635. Retrying in 10 seconds.


Retrieving sound features:  24%|██▍       | 99/415 [01:10<00:34,  9.19it/s]  

Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.


Retrieving sound features:  24%|██▍       | 99/415 [01:26<00:34,  9.19it/s]

Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retrying in 10 seconds.
Feature extraction failed for sound 359183. Retr

KeyboardInterrupt: 

In [69]:
# force_mkdir(OGG_PATH)
# force_mkdir(WAV_PATH)

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)

sound_list = create_sound_list(FREESOUND_QUERIES, freesound_client)
sound_dict = create_sound_dict(FREESOUND_QUERIES, freesound_client)


# download_sound_previews(sound_list, OGG_PATH, freesound_client)

wav_filenames, total_length = ogg_to_wav(OGG_PATH, WAV_PATH)


[wav @ 0x75438740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x7589c840] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x751de620] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x7589c840] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x7589c840] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x7589c840] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x7589c840] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x7589c840] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

In [4]:
FEATURES = ["lowlevel.pitch.mean",
            "rhythm.beats_count",
            "rhythm.onset_rate",
            "rhythm.beats_loudness.mean",
            "rhythm.beats_loudness.var",
            "sfx.logattacktime.mean",
            ]
for feature in FEATURES:
    print(eval(f"sound_list[0].get_analysis().{feature}"))

214.62378643195854
4.999997509048382
0.33128004699993613
0.00815768361060615
4.61615248472025e-05
-0.5570832191204728


In [ ]:
sound_list[0].get_analysis()

In [15]:
get_features(sound_list[2], FEATURES)

[316.9854480448869,
 8.00000284680185,
 0.5685334181077539,
 0.014363767435153552,
 0.0001692855524310017,
 -0.4918138700097916]

In [32]:
a = np.vstack([get_features(sound, FEATURES) for sound in sound_list[:35]])

FreesoundException: <FreesoundException: code=429, detail="Too Many Requests">

In [28]:
a

array([[ 2.14623786e+02,  4.99999751e+00,  3.31280047e-01,
         8.15768361e-03,  4.61615248e-05, -5.57083219e-01],
       [ 3.25092774e+02,  5.99999929e+00,  5.68533418e-01,
         6.64459649e-03,  2.63845353e-05, -7.54464891e-01],
       [ 3.16985448e+02,  8.00000285e+00,  5.68533418e-01,
         1.43637674e-02,  1.69285552e-04, -4.91813870e-01],
       [ 3.26548847e+02,  4.99999751e+00,  5.68533418e-01,
         4.73608653e-03,  1.49611271e-05, -2.44246051e-01],
       [ 4.84479973e+02,  7.00000107e+00,  7.46816206e-01,
         1.09512077e-02,  9.69449082e-05, -3.66691844e-01],
       [ 3.15343222e+02,  5.99999929e+00,  3.03284541e-01,
         5.46519340e-03,  2.91804744e-05, -7.32375978e-01],
       [ 3.08590432e+02,  7.00000107e+00,  3.06522488e-01,
         1.18529485e-02,  7.79439344e-05, -5.00238121e-01],
       [ 3.19449398e+02,  5.99999929e+00,  6.52521182e-01,
         1.69471850e-02,  1.57919204e-04, -1.80157724e-01],
       [ 4.21636596e+02,  4.99999751e+00,  6.702

In [29]:
pd.isna(a).any()

True

In [26]:
a

array([[ 2.14623786e+02,  4.99999751e+00,  3.31280047e-01,
         8.15768361e-03,  4.61615248e-05, -5.57083219e-01],
       [ 3.25092774e+02,  5.99999929e+00,  5.68533418e-01,
         6.64459649e-03,  2.63845353e-05, -7.54464891e-01],
       [ 3.16985448e+02,  8.00000285e+00,  5.68533418e-01,
         1.43637674e-02,  1.69285552e-04, -4.91813870e-01],
       [ 3.26548847e+02,  4.99999751e+00,  5.68533418e-01,
         4.73608653e-03,  1.49611271e-05, -2.44246051e-01],
       [ 4.84479973e+02,  7.00000107e+00,  7.46816206e-01,
         1.09512077e-02,  9.69449082e-05, -3.66691844e-01],
       [ 3.15343222e+02,  5.99999929e+00,  3.03284541e-01,
         5.46519340e-03,  2.91804744e-05, -7.32375978e-01],
       [ 3.08590432e+02,  7.00000107e+00,  3.06522488e-01,
         1.18529485e-02,  7.79439344e-05, -5.00238121e-01],
       [ 3.19449398e+02,  5.99999929e+00,  6.52521182e-01,
         1.69471850e-02,  1.57919204e-04, -1.80157724e-01],
       [ 4.21636596e+02,  4.99999751e+00,  6.702

In [72]:

train, test = generate_train_test_dataframes(sound_dict, sound_list, WAV_PATH, DAC_PATH, FEATURES)

train_path = os.path.join(DAC_PATH, "dac-train")
val_path = os.path.join(DAC_PATH, "dac-val")

force_mkdir(train_path)
force_mkdir(val_path)

encode_audio_files(train, train_path, model_sr='44khz', model_tag='latest')
encode_audio_files(test, val_path, model_sr='44khz', model_tag='latest')

<FreesoundException: code=429, detail="Too Many Requests">
Feature extraction failed for sound 359649. Retrying in 10 seconds.


KeyboardInterrupt: 

In [60]:
FREESOUND_STORE_METADATA_FIELDS = ['name']

In [78]:
pager = freesound_client.text_search(
        query = FREESOUND_QUERIES[0]['query'],
        filter = 'pack:"Flute single notes (played badly)"',
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 0,
        page_size = 1
    )

In [84]:
pager

TypeError: 'dict' object is not callable

In [31]:
train, test = generate_train_test_dataframes(sound_dict, sound_list, WAV_PATH, DAC_PATH, FEATURES)

(10,) (0, 10) 246761
(10,) (1, 10) 357356
(10,) (2, 10) 357703
(10,) (3, 10) 360251
(10,) (4, 10) 247391
(10,) (5, 10) 354624
(10,) (6, 10) 355014
(10,) (7, 10) 359763
(10,) (8, 10) 357734
(10,) (9, 10) 357703
(10,) (10, 10) 358111
(10,) (11, 10) 354416
(10,) (12, 10) 359640
(10,) (13, 10) 357703
(10,) (14, 10) 354836
(10,) (15, 10) 360055
(10,) (16, 10) 357734
(10,) (17, 10) 247469
(10,) (18, 10) 361080
(10,) (19, 10) 361080


ValueError: The test_size = 4 should be greater or equal to the number of classes = 5

In [18]:
from pathlib import Path
from dac.utils import load_model
from audiotools import AudioSignal
import torch

def encode_audio_files(df, output_dir, model_sr='24khz', model_tag='latest', device="cuda"):
    with torch.no_grad():
        # Load the DAC model
        model = load_model(model_type=model_sr, tag=model_tag).to(device)
        model.eval()


        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Iterate over all WAV files
        for file_path in tqdm(df["Audio ID"]):

            wav_path = df[df["Audio ID"] == file_path]["WAV Path"].values[0]

            # Load the audio file
            audio = AudioSignal(wav_path)

            # Extract the tensor data and move it to the appropriate device
            audio_tensor = audio.audio_data.to(device)

            # Encode the audio tensor
            codes = model.encode(audio_tensor)

            # Define the output file path
            output_file = os.path.join(output_dir, file_path + '.dac')

            # Save the encoded codes
            with open(output_file, 'wb') as f:
                torch.save(codes, f)

train_path = os.path.join(DAC_PATH, "dac-train")
val_path = os.path.join(DAC_PATH, "dac-val")

force_mkdir(train_path)
force_mkdir(val_path)

encode_audio_files(train, train_path, model_sr='44khz', model_tag='latest')
encode_audio_files(test, val_path, model_sr='44khz', model_tag='latest')

/home/angel/anaconda3/envs/valerio/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
100%|██████████| 9/9 [00:03<00:00,  2.64it/s]


In [8]:
torch.cuda.empty_cache()

In [ ]:

def convert_to_dac(wav_filenames, output_folder):
    for filename in wav_filenames:
        a = os.system(f"python3 -m dac encode wavs/\{filename} --model_bitrate 8kbps --n_quantizers 4 --output {output_folder}")


train_path = os.path.join(DAC_PATH, "dac-train")
val_path = os.path.join(DAC_PATH, "dac-val")

force_mkdir(train_path)
force_mkdir(val_path)

convert_to_dac(train["WAV Path"], train_path)
convert_to_dac(test["WAV Path"], val_path)


In [ ]:
def convert_to_dac(wav_filenames, output_folder):
    for filename in wav_filenames:
        a = os.system(f"python3 -m dac encode {filename} --model_bitrate 8kbps --n_quantizers 4 --output {output_folder}")


convert_to_dac(wav_filenames, DAC_PATH)


In [146]:
from sklearn.model_selection import train_test_split

def create_train_test_dataframes(sound_dict, wav_filenames, wav_path, dac_folder):

    df = pd.DataFrame.from_dict(generate_class_mapping(sound_dict, wav_filenames), orient="index", columns=["Class Name"]).reset_index(names="Audio ID")
    df["WAV Path"] = df["Audio ID"].apply(lambda x: os.path.join(wav_path, x) + ".wav")
    df["DAC Path"] = df["Audio ID"].apply(lambda x: os.path.join(dac_folder, x) + ".dac")
    train, test = train_test_split(df, test_size=0.2, stratify=df["Class Name"])

    train.to_excel(os.path.join(dac_folder, "dac-train.xlsx"))
    test.to_excel(os.path.join(dac_folder, "dac-val.xlsx"))



In [145]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, stratify=df["Class Name"])
train

,Audio ID,Class Name,wav_path
434,511348_220500,rain,wavs/511348_220500.wav
107,103325_220500,helicopter,wavs/103325_220500.wav
245,435676_661500,helicopter,wavs/435676_661500.wav
173,165935_0,rain,wavs/165935_0.wav
325,467225_661500,helicopter,wavs/467225_661500.wav
...,...,...,...
441,250162_220500,radio,wavs/250162_220500.wav
172,70606_0,radio,wavs/70606_0.wav
111,432346_0,radio,wavs/432346_0.wav
291,346372_220500,helicopter,wavs/346372_220500.wav


* Crear lista de nombres de sonidos
* Crear un mapping de nombres de sonidos a clases
* 

In [5]:
freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
sounds = sum([query_freesound(query['query'], query['filter'], freesound_client, query['num_results']) for query in FREESOUND_QUERIES],[])


In [31]:
a = sounds[0].get_analysis()
a.as_dict().keys()

dict_keys(['lowlevel', 'tonal', 'sfx', 'rhythm', 'metadata'])

In [45]:

features = ["lowlevel.pitch.mean",
            "rhythm.beats_count",
            "rhythm.onset_rate",
            "rhythm.beats_loudness.mean",
            "rhythm.beats_loudness.var",
            "sfx.logattacktime.mean",
            ]

def get_features(sound: freesound.FreesoundObject, features: list):
    try:
        a = sound.get_analysis()
        feat_vector = list()
        for feature in features:
            feat_vector.append(eval(f"a.{feature}"))
    except:
        return [np.nan for f in features]

In [62]:
sounds

[<Sound: id="49481", name="helicopterPassage_28sec.wav">,
 <Sound: id="331228", name="Helicopter following me.mp3">,
 <Sound: id="561819", name="Helicopter Overhead SFX.wav">,
 <Sound: id="543127", name="Electric Helicopter flying around.wav">,
 <Sound: id="655098", name="Helicopter.wav">,
 <Sound: id="741812", name="Helicopter">,
 <Sound: id="365829", name="Helicopter Vocalization">,
 <Sound: id="435676", name="2018 - Helicopters - Waalre NL">,
 <Sound: id="670111", name="Helicopter Passover">,
 <Sound: id="730294", name="helicopter">,
 <Sound: id="507194", name="Helicopter, Short Version.wav">,
 <Sound: id="442954", name="Helicopter Mouth">,
 <Sound: id="563901", name="HELICOPTER_EU.wav">,
 <Sound: id="514921", name="helicopter, ecuriel b2 helicopter, exit and walk away from idling helicopter">,
 <Sound: id="513397", name="Helicopter Flying Overhead.wav">,
 <Sound: id="133500", name="circling helicopter m97_34c">,
 <Sound: id="494829", name="Helicopter overhead. Venice Beach, LA. 201

In [54]:
sound_analysis = sounds[0].get_analysis()
for feature in features:
    eval(f"sound_analysis.{feature}")

In [40]:
a.as_dict()["lowlevel.pitch.mean"]

KeyError: 'lowlevel.pitch.mean'

# Download from freesound

In [60]:
freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)

force_mkdir(FILES_DIR)

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], freesound_client, query['num_results']) for query in FREESOUND_QUERIES],[])

sound_dict = {query["query"]: query_freesound(query['query'], query['filter'], freesound_client, query['num_results']) for query in FREESOUND_QUERIES}

mapping = dict()
# Reverse sound_dict
for key, value in sound_dict.items():
    for s in value:
        mapping[s.name.replace(" ", "_").replace("/", "")] = key


# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(tqdm(sounds, desc='Downloading sounds')):
    retrieve_sound_preview(sound, freesound_client, FILES_DIR)

In [61]:
sound_dict

{'helicopter': [<Sound: id="49481", name="helicopterPassage_28sec.wav">,
  <Sound: id="331228", name="Helicopter following me.mp3">,
  <Sound: id="561819", name="Helicopter Overhead SFX.wav">,
  <Sound: id="543127", name="Electric Helicopter flying around.wav">,
  <Sound: id="655098", name="Helicopter.wav">,
  <Sound: id="741812", name="Helicopter">,
  <Sound: id="365829", name="Helicopter Vocalization">,
  <Sound: id="435676", name="2018 - Helicopters - Waalre NL">,
  <Sound: id="670111", name="Helicopter Passover">,
  <Sound: id="730294", name="helicopter">,
  <Sound: id="507194", name="Helicopter, Short Version.wav">,
  <Sound: id="442954", name="Helicopter Mouth">,
  <Sound: id="563901", name="HELICOPTER_EU.wav">,
  <Sound: id="514921", name="helicopter, ecuriel b2 helicopter, exit and walk away from idling helicopter">,
  <Sound: id="513397", name="Helicopter Flying Overhead.wav">,
  <Sound: id="133500", name="circling helicopter m97_34c">,
  <Sound: id="494829", name="Helicopter 

In [70]:
sound_dict
names = sound_dict.values()

mapping = dict()
for class_name, sound_list in sound_dict:
    for s in sound_list:
        mapping[s] = 

{[v.name: key for ] for key, value in sound_dict.items()}

AttributeError: 'list' object has no attribute 'name'

In [71]:
sound_dict

{'helicopter': [<Sound: id="49481", name="helicopterPassage_28sec.wav">,
  <Sound: id="331228", name="Helicopter following me.mp3">,
  <Sound: id="561819", name="Helicopter Overhead SFX.wav">,
  <Sound: id="543127", name="Electric Helicopter flying around.wav">,
  <Sound: id="655098", name="Helicopter.wav">,
  <Sound: id="741812", name="Helicopter">,
  <Sound: id="365829", name="Helicopter Vocalization">,
  <Sound: id="435676", name="2018 - Helicopters - Waalre NL">,
  <Sound: id="670111", name="Helicopter Passover">,
  <Sound: id="730294", name="helicopter">,
  <Sound: id="507194", name="Helicopter, Short Version.wav">,
  <Sound: id="442954", name="Helicopter Mouth">,
  <Sound: id="563901", name="HELICOPTER_EU.wav">,
  <Sound: id="514921", name="helicopter, ecuriel b2 helicopter, exit and walk away from idling helicopter">,
  <Sound: id="513397", name="Helicopter Flying Overhead.wav">,
  <Sound: id="133500", name="circling helicopter m97_34c">,
  <Sound: id="494829", name="Helicopter 

# Convert to WAV (5s long)

In [66]:

import essentia.standard as es
import os


force_mkdir("wavs")
mapping2 = dict()
total_length = 0

import pandas as pd
meta = pd.read_csv("/home/angel/Downloads/2529934/FSDnoisy18k.meta/train.csv")
meta = meta[meta.manually_verified == 1]

for filename in os.listdir("oggs"):
    ogg_path = os.path.join("oggs", filename)
    # Change extension to WAV
    wav_path = os.path.join("wavs", filename)

    # Load OGG file using Essentia
    audio = es.MonoLoader(filename=ogg_path)()
    
    # Trim to first 5 seconds (Essentia loads audio as a NumPy array with a sample rate)
    sample_rate = 44100  # Default sample rate in Essentia

    total_length += len(audio) / sample_rate
    # Trim audio into 5s frames:
    for i in range(0, len(audio), 5 * sample_rate):
        trimmed_audio = audio[i:i + 5 * sample_rate]
        if len(trimmed_audio) < 5 * sample_rate:
            continue
        # Save as WAV. include i right before the extension
        path = wav_path + f"_{i}.wav"
        
        es.MonoWriter(filename=path, format='wav')(trimmed_audio)
        mapping2[path] = mapping[filename]
        print(f"Processed: {filename} -> {wav_path}")



Processed: Lo-fi_AMFM_radio_(Two_stations_mixed).wav -> wavs/Lo-fi_AMFM_radio_(Two_stations_mixed).wav
Processed: urban_night_rain_1 -> wavs/urban_night_rain_1
Processed: urban_night_rain_1 -> wavs/urban_night_rain_1
Processed: urban_night_rain_1 -> wavs/urban_night_rain_1
Processed: urban_night_rain_1 -> wavs/urban_night_rain_1
Processed: airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background -> wavs/airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background
Processed: airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background -> wavs/airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background
Processed: airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background -> wavs/airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background
Processed: Roland_Super_Saw_Staccato_C.wav -> wavs/Roland_Super_Sa

[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

Processed: Dripping_rain.wav -> wavs/Dripping_rain.wav
Processed: Dripping_rain.wav -> wavs/Dripping_rain.wav
Processed: Dripping_rain.wav -> wavs/Dripping_rain.wav
Processed: baby_radio_1.mp3 -> wavs/baby_radio_1.mp3
Processed: radio11.wav -> wavs/radio11.wav
Processed: radio11.wav -> wavs/radio11.wav
Processed: Rain_closed_window -> wavs/Rain_closed_window
Processed: Rain_closed_window -> wavs/Rain_closed_window
Processed: Rain_closed_window -> wavs/Rain_closed_window
Processed: Rain_closed_window -> wavs/Rain_closed_window
Processed: Radio_Noise_&_Mod_Rez_2.aif -> wavs/Radio_Noise_&_Mod_Rez_2.aif
Processed: Radio_Noise_&_Mod_Rez_2.aif -> wavs/Radio_Noise_&_Mod_Rez_2.aif
Processed: old-radio-noise-switchon-ukw.wav -> wavs/old-radio-noise-switchon-ukw.wav
Processed: old-radio-noise-switchon-ukw.wav -> wavs/old-radio-noise-switchon-ukw.wav
Processed: old-radio-noise-switchon-ukw.wav -> wavs/old-radio-noise-switchon-ukw.wav
Processed: room-tone_wind_rain_10_200216_0114.wav -> wavs/room-

[wav @ 0x448b0fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448b0fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448b0fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ac1fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ac1fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ac1fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448b0fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448b0fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

Processed: Taste_Bass_Supersaw_Synth_F#_4_Measures -> wavs/Taste_Bass_Supersaw_Synth_F#_4_Measures
Processed: Taste_Bass_Supersaw_Synth_F#_4_Measures -> wavs/Taste_Bass_Supersaw_Synth_F#_4_Measures
Processed: Taste_Bass_Supersaw_Synth_F#_4_Measures -> wavs/Taste_Bass_Supersaw_Synth_F#_4_Measures
Processed: airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead -> wavs/airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead
Processed: airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead -> wavs/airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead
Processed: airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead -> wavs/airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead
Processed: Helicopter_Overhead_SFX.wav -> wavs/Helicopter_Overhead_SFX.wav
Processed: Rain_Drips_in_a_bucket.mp3 -> wavs/Rain_Drips_in_a_bucket.mp3
Processed: Rain_Drips_in_a_bucket.mp3 -> wavs/Rain_Drips_in_a_bucket.mp3
Processed: Rain_Drips_i

...

[wav @ 0x4356f360] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4356f360] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4356f360] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ab1600] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ab1600] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ab1600] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448a2fc0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448a2fc0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

Processed: OH-58_Helicopter_Loop.flac -> wavs/OH-58_Helicopter_Loop.flac
Processed: rain.wav -> wavs/rain.wav
Processed: Helicopter_over_the_beach_today.mp3 -> wavs/Helicopter_over_the_beach_today.mp3
Processed: Helicopter_over_the_beach_today.mp3 -> wavs/Helicopter_over_the_beach_today.mp3
Processed: Helicopter_over_the_beach_today.mp3 -> wavs/Helicopter_over_the_beach_today.mp3
Processed: 1977_events_01.mp3 -> wavs/1977_events_01.mp3
Processed: Rain-Light-Loopable.wav -> wavs/Rain-Light-Loopable.wav
Processed: Short_Rain.wav -> wavs/Short_Rain.wav
Processed: Short_Rain.wav -> wavs/Short_Rain.wav
Processed: Various_Helicopter_Loops -> wavs/Various_Helicopter_Loops
Processed: Various_Helicopter_Loops -> wavs/Various_Helicopter_Loops
Processed: Various_Helicopter_Loops -> wavs/Various_Helicopter_Loops


[wav @ 0x4488fba0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4357aea0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4357aea0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4357aea0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4357aea0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4488fba0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4357aea0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x447b8d60] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

Processed: rain_-_gutter_-_outside_laudry_room_window.wav -> wavs/rain_-_gutter_-_outside_laudry_room_window.wav
Processed: rain_-_gutter_-_outside_laudry_room_window.wav -> wavs/rain_-_gutter_-_outside_laudry_room_window.wav
Processed: rain_-_gutter_-_outside_laudry_room_window.wav -> wavs/rain_-_gutter_-_outside_laudry_room_window.wav
Processed: rain_-_gutter_-_outside_laudry_room_window.wav -> wavs/rain_-_gutter_-_outside_laudry_room_window.wav
Processed: rain_-_gutter_-_outside_laudry_room_window.wav -> wavs/rain_-_gutter_-_outside_laudry_room_window.wav
Processed: rain_against_window -> wavs/rain_against_window
Processed: rain_against_window -> wavs/rain_against_window
Processed: rain_against_window -> wavs/rain_against_window
Processed: rain_against_window -> wavs/rain_against_window
Processed: RAIN_STICK_A_001.wav -> wavs/RAIN_STICK_A_001.wav
Processed: radio_in_room.wav -> wavs/radio_in_room.wav
Processed: radio_in_room.wav -> wavs/radio_in_room.wav
Processed: radio_in_room.wav

[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4484ac80] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

# Convert to DAC

In [33]:

force_mkdir("testdata/dac-train")
force_mkdir("testdata/dac-val")
a = os.system("python3 -m dac encode wavs --model_bitrate 8kbps --n_quantizers 4 --output testdata/dac-train")
a = os.system("python3 -m dac encode wavs_test --model_bitrate 8kbps --n_quantizers 4 --output testdata/dac-val")
print(a)
print("Total length: ", total_length, " seconds of audio (", total_length / 3600, " hours)")


2025-03-20 00:53:45.921917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742428425.934850   47018 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742428425.938620   47018 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-20 00:53:45.952061: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/angel/anaconda3/envs/valerio/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.u

0
Total length:  2735.360045351476  seconds of audio ( 0.7598222348198544  hours)


# Generate train and validation excels

In [27]:
entries = list()
for filename, class_name in mapping2.items():
    filename = os.path.basename(filename)
    filename = filename.replace(" ", "_")
    filename = filename[:-4]
    entries.append({"Full File Name": f"{filename}" + ".dac", 
                    "Class Name": f"{class_name}",
                    "Param1": 1,
                    "Param2": 2,
                    "Param3": 3})
    

pd.DataFrame(entries).to_excel("testdata/dac-train.xlsx")
pd.DataFrame(entries).to_excel("testdata/dac-val.xlsx")

In [4]:
import os

In [6]:
import os
os.path.exists("/home/angel/Downloads/2529934/FSDnoisy18k.audio_test")

True

In [8]:
import pandas as pd
import essentia.standard as es

force_mkdir("wavs")
force_mkdir("wavs_test")

mapping2 = dict()
total_length = 0

meta = pd.read_csv("/home/angel/Downloads/2529934/FSDnoisy18k.meta/train.csv")
meta = meta[(meta.label.isin(["Dishes_and_pots_and_pans", "Rain", "Fireworks", "Writing"]))]# & (meta.manually_verified == 1)]

for filename in meta.fname:
    ogg_path = os.path.join("/home/angel/Downloads/2529934/FSDnoisy18k.audio_train", filename)
    # Change extension to WAV
    wav_path = os.path.join("wavs", filename)

    # Load OGG file using Essentia
    audio = es.MonoLoader(filename=ogg_path)()
    
    # Trim to first 5 seconds (Essentia loads audio as a NumPy array with a sample rate)
    sample_rate = 44100  # Default sample rate in Essentia

    total_length += len(audio) / sample_rate
    # Trim audio into 5s frames:
    for i in range(0, len(audio), 5 * sample_rate):
        trimmed_audio = audio[i:i + 5 * sample_rate]
        if len(trimmed_audio) < 5 * sample_rate:
            continue
        # Save as WAV. include i right before the extension
        path = wav_path + f"_{i}.wav"
        
        es.MonoWriter(filename=path, format='wav')(trimmed_audio)
        mapping2[path] = meta.label[meta.fname == filename].values[0]
        #print(f"Processed: {filename} -> {wav_path}")

meta = pd.read_csv("/home/angel/Downloads/2529934/FSDnoisy18k.meta/test.csv")
meta = meta[meta.label.isin(["Dishes_and_pots_and_pans", "Rain", "Fireworks", "Writing"])]

for filename in meta.fname:
    ogg_path = os.path.join("/home/angel/Downloads/2529934/FSDnoisy18k.audio_test", filename)
    # Change extension to WAV
    wav_path = os.path.join("wavs_test", filename)

    # Load OGG file using Essentia
    audio = es.MonoLoader(filename=ogg_path)()
    
    # Trim to first 5 seconds (Essentia loads audio as a NumPy array with a sample rate)
    sample_rate = 44100  # Default sample rate in Essentia

    total_length += len(audio) / sample_rate
    # Trim audio into 5s frames:
    for i in range(0, len(audio), 5 * sample_rate):
        trimmed_audio = audio[i:i + 5 * sample_rate]
        if len(trimmed_audio) < 5 * sample_rate:
            continue
        # Save as WAV. include i right before the extension
        path = wav_path + f"_{i}.wav"
        
        es.MonoWriter(filename=path, format='wav')(trimmed_audio)
        mapping2[path] = meta.label[meta.fname == filename].values[0]
        #print(f"Processed: {filename} -> {wav_path}")

[wav @ 0x2adf51e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x2ae00880] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x2ae00880] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x2ae00880] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x2adf51e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x2ae4de60] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x2ae4de60] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x2ae4de60] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

In [19]:

force_mkdir("testdata/dac-train")
force_mkdir("testdata/dac-val")
a = os.system("python3 -m dac encode wavs --model_bitrate 8kbps --n_quantizers 4 --output testdata/dac-train")
a = os.system("python3 -m dac encode wavs_test --model_bitrate 8kbps --n_quantizers 4 --output testdata/dac-val")
print(a)
print("Total length: ", total_length, " seconds of audio (", total_length / 3600, " hours)")


2025-03-22 16:26:12.953035: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742657172.979096   76747 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742657172.986553   76747 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/home/angel/anaconda3/envs/valerio/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Encoding files: 100%|██████████| 43/43 [00:18<00:00,  2.31it/s]
2025-03-22 16:26:40.826708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT fac

0
Total length:  263.64816326530615  seconds of audio ( 0.07323560090702949  hours)


In [13]:
entries = list()
for filename in os.listdir("wavs"):
    full_filename = os.path.join("wavs", filename)
    class_name = mapping2[full_filename]
    #filename = os.path.basename(filename)
    filename = filename.replace(" ", "_")
    filename = filename[:-4]
    entries.append({"Full File Name": f"{filename}" + ".dac", 
                    "Class Name": f"{class_name}",
                    "Param1": 1,
                    "Param2": 2,
                    "Param3": 3})
    

entries = pd.DataFrame(entries)[:50].to_excel("testdata/dac-train.xlsx")

entries = list()
for filename in os.listdir("wavs_test"):
    full_filename = os.path.join("wavs_test", filename)
    class_name = mapping2[full_filename]
    #filename = os.path.basename(filename)
    filename = filename.replace(" ", "_")
    filename = filename[:-4]
    entries.append({"Full File Name": f"{filename}" + ".dac", 
                    "Class Name": f"{class_name}",
                    "Param1": 1,
                    "Param2": 2,
                    "Param3": 3})
pd.DataFrame(entries).to_excel("testdata/dac-val.xlsx")

In [8]:
meta = pd.read_csv("/home/angel/Downloads/2529934/FSDnoisy18k.meta/train.csv")
meta.label.unique()

array(['Walk_or_footsteps', 'Coin_(dropping)', 'Dishes_and_pots_and_pans',
       'Bass_guitar', 'Crash_cymbal', 'Slam', 'Clapping', 'Rain', 'Wind',
       'Piano', 'Engine', 'Glass', 'Fire', 'Acoustic_guitar', 'Fart',
       'Fireworks', 'Hi-hat', 'Squeak', 'Tearing', 'Writing'],
      dtype=object)

In [1]:
meta

NameError: name 'meta' is not defined

mapping